In [1]:
from bnn import BNN
from cem_optimizer_v2 import CEM_opt
from Propagation import Propagation_net
from collections import deque
import torch
import numpy as np

DEVICE = 'cuda:0' if torch.cuda.is_available() else 'cpu'
torch.set_default_dtype(torch.float64)

In [2]:
horizon = 10

cem = CEM_opt(
    num_action_seq=20,
    action_seq_len=4 * horizon, 
    percent_elite=0.1
)



In [3]:
from metaworld.envs import (ALL_V2_ENVIRONMENTS_GOAL_OBSERVABLE)

env = ALL_V2_ENVIRONMENTS_GOAL_OBSERVABLE['reach-wall-v2-goal-observable']()    
s = env.reset()

/home/dema/PycharmProjects/lifelong_rl/venv/lib/python3.10/site-packages/gym/spaces/box.py:127: UserWarning: WARN: Box bound precision lowered by casting to float64
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")
/home/dema/PycharmProjects/lifelong_rl/venv/lib/python3.10/site-packages/gym/spaces/box.py:127: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


In [4]:
def apply(act_seq, s, env):
    act_seq = act_seq.reshape((-1, 4))
    cumulate_rew = 0
    for act in act_seq: 
        s_prime, r, d, info = env.step(act)
        s = s_prime
        cumulate_rew += r
    return cumulate_rew/act_seq.shape[0]

def apply_rand(act_seq, s, env):
    act_seq = act_seq.reshape((-1, 4))
    cumulate_rew = 0
    for act in act_seq: 
        s_prime, r, d, info = env.step(act + np.random.randn(4)*0.5)
        s = s_prime
        cumulate_rew += r
    return cumulate_rew/act_seq.shape[0]


act_sequence = cem.population
s = env.reset()
apply(act_sequence[0], s, env) # DETERMINISTIC

1.3092079897672715

In [5]:
reference_act_seq = act_sequence[0]

In [89]:
reward_for_act_seq = []
for particle in range(20):
    s = env.reset()
    avg_reward = apply_rand(reference_act_seq, s, env)
    reward_for_act_seq.append(avg_reward)

sum(reward_for_act_seq)/len(reward_for_act_seq)

1.3062740872594738

In [100]:
propagation_net = Propagation_net(
            num_particles=20,
            action_dim=4,
            obs_dim=39
        )
propagation_net.move_to_gpu()
dynamic_dict = torch.load('model_stock/dynamic_small_evn_25_lr0.001_3000EP.pth')
dynamic_net = BNN(action_dim=4, obs_dim=39, reward_dim=1, weight=dynamic_dict).to(DEVICE)


In [157]:
propagation_net.sample_from(dynamic_net)
propagation_net.propagate(torch.from_numpy(s).to(DEVICE), torch.from_numpy(reference_act_seq).to(DEVICE))    

tensor(3.3926, device='cuda:0')